# Train the Grader

In [1]:
import torch.utils.data as Data
import torch

data = torch.load(
    "Data/Beat_dataset/Beat_tensor_dataset/beat_no_head_rsz_75.pt")
x = data['x']
y = data['y']
batch_size = 64
dataset = Data.TensorDataset(x, y)
train_iter = Data.DataLoader(dataset, batch_size, shuffle=True, num_workers=0)

In [2]:
from GenMusic.models.mlp import MLP

mlp = MLP(75, [150, 50, 10], 2)

In [3]:
from torch import nn

lr = 1e-3
epoch = 1000
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(mlp.parameters(), lr=lr)
device = torch.device("cuda")
torch.cuda.empty_cache()  # 释放显存

In [ ]:
from GenMusic.models import train

model = train(train_iter, mlp, criterion,
              optimizer, epoch, device, autoPrint=0)

# Generate beat

In [1]:
import torch
from GenMusic.bin.rewarders import nngrader_rewarder, get_nngrader_rewarder
from GenMusic import ga_generator

rwd = get_nngrader_rewarder("runs/1708865989.470388/best.pt")
rwd.set_tgt(torch.tensor([0.5, 0.5]))
ga = ga_generator(rwd)

In [2]:
mxomin = [1]
dim = 48
varTypes = [0]*48
lb = [0.3]*48
ub = [0.9]*48
lbin = [1]*48
ubin = [1]*48
ga.set_problem_params(maxormins=mxomin, Dim=dim,
                      varTypes=varTypes, lb=lb, ub=ub, lbin=lbin, ubin=lbin)
ga.__set_problem_aimFunc(ga.__tensor_with_grad)
ga.set_population_params()
ga.set_algorithm_params()

In [3]:
ga.generate_ori()
print(ga.best_tensor)

# midi GM

In [5]:
import torch
from GenMusic.bin.rewarders import nngrader_rewarder, get_nngrader_rewarder, sad_simple_rewarder,demo_dtw_midi_rewarder
from GenMusic import ga_generator

rwd = demo_dtw_midi_rewarder("Data/mini_midi/npy_file/happy","Data/mini_midi/npy_file/sad","Data/covInv.npy")

ga = ga_generator(rwd)


In [6]:
l = 9
mxomin = [1]
dim = 4*l
varTypes = [0]*dim
lb = [0]*dim
ub = [127]*dim
ub[l:2*l] = [250]*l
lbin = [1]*dim
ubin = [1]*dim
ga.set_tgt("sad")
ga.set_problem_params(maxormins=mxomin, Dim=dim,
                      varTypes=varTypes, lb=lb, ub=ub, lbin=lbin, ubin=lbin)
ga.set_population_params()
ga.set_algorithm_params()

ga.generate_ori()

In [4]:
from tools.feature2midi import get_midifile
get_midifile((torch.reshape(ga.best_tensor,(4,-1)).T).numpy())